In [1]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline

# Load pre-trained BERT model and tokenizer for NER
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = BertForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Create NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Sample text
text = "Elon Musk is the CEO of SpaceX, which is based in Hawthorne, California."

# Run NER
results = ner_pipeline(text)

# Display results
for entity in results:
    print(f"Entity: {entity['word']} - Label: {entity['entity']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Entity: El - Label: I-PER
Entity: ##on - Label: I-PER
Entity: Mu - Label: I-PER
Entity: ##sk - Label: I-PER
Entity: Space - Label: I-ORG
Entity: ##X - Label: I-ORG
Entity: Hawthorne - Label: I-LOC
Entity: California - Label: I-LOC


In [4]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline

# Load pre-trained BERT model and tokenizer for NER
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = BertForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Create NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Sample text
text = "Barack Obama was the 44th President of the United States."

# Run NER
results = ner_pipeline(text)

# Display results
for entity in results:
    print(f"Entity: {entity['word']} - Label: {entity['entity']}")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Entity: Barack - Label: I-PER
Entity: Obama - Label: I-PER
Entity: United - Label: I-LOC
Entity: States - Label: I-LOC


In [2]:
import spacy

# Load spaCy's transformer-based NER pipeline
nlp = spacy.load("en_core_web_trf")  # Transformer-based pipeline

# Sample text
text = "Apple Inc. is a technology company headquartered in Cupertino, California."

# Process text
doc = nlp(text)

# Display results
for ent in doc.ents:
    print(f"Entity: {ent.text} - Label: {ent.label_}")

Entity: Apple Inc. - Label: ORG
Entity: Cupertino - Label: GPE
Entity: California - Label: GPE


In [10]:
import spacy
from spacy.training import Example
import pandas as pd

# Load your custom dataset
data = pd.read_csv("healthcare_dataset.csv")

# Print the column names to see what's available
print("Column Names:", data.columns)

# Display the first few rows of the dataset to understand its structure
print(data.head())

# After examining the output, manually specify the correct column names
# Replace 'Name' and 'Entities' with the actual column names from your dataset
text_column_name = 'Name'  # Replace with the actual column name for text
entity_column_name = 'Entities'  # Replace with the actual column name for entities

# Check if the specified column names are correct
if text_column_name not in data.columns:
    raise ValueError(f"'{text_column_name}' column not found in the dataset.")
if entity_column_name not in data.columns:
    raise ValueError(f"'{entity_column_name}' column not found in the dataset.")

# Load spaCy's blank model
nlp = spacy.blank("en")

# Define a simple NER pipeline
ner = nlp.add_pipe("ner")

# Add labels to the NER model (example labels)
ner.add_label("DISEASE")
ner.add_label("CHEMICAL")

# Prepare training data
train_data = []
for _, row in data.iterrows():
    text = row[text_column_name]
    entities = row[entity_column_name]  # Ensure this column contains entities as a list of tuples
    train_data.append((text, {"entities": entities}))

# Training the model
optimizer = nlp.begin_training()
for text, annotations in train_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    nlp.update([example], sgd=optimizer, drop=0.5, losses={})

# Test the model
test_text = "Patient was diagnosed with breast cancer and was given Tamoxifen."
doc = nlp(test_text)
for ent in doc.ents:
    print(f"Entity: {ent.text} - Label: {ent.label_}")

Column Names: Index(['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition',
       'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider',
       'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date',
       'Medication', 'Test Results'],
      dtype='object')
            Name  Age  Gender Blood Type Medical Condition Date of Admission  \
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31   
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22   
3   andrEw waTtS   28  Female         O+          Diabetes        2020-11-18   
4  adrIENNE bEll   43  Female        AB+            Cancer        2022-09-19   

             Doctor                    Hospital Insurance Provider  \
0     Matthew Smith             Sons and Miller         Blue Cross   
1   Samantha Davies                     Kim Inc           Medicare   
2  Tiffany Mitchell 

ValueError: 'Entities' column not found in the dataset.

In [14]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline
from datasets import load_dataset

# Load a specific configuration of the dataset
dataset = load_dataset('financial_phrasebank', 'sentences_allagree')

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForTokenClassification.from_pretrained("bert-base-uncased")

# Create NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Sample text from the financial dataset
text = "The stock price of Apple Inc. rose by 5%."

# Run NER
results = ner_pipeline(text)

# Display results
for entity in results:
    print(f"Entity: {entity['word']} - Label: {entity['entity']}")

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Entity: the - Label: LABEL_0
Entity: stock - Label: LABEL_0
Entity: price - Label: LABEL_0
Entity: of - Label: LABEL_1
Entity: apple - Label: LABEL_0
Entity: inc - Label: LABEL_0
Entity: . - Label: LABEL_1
Entity: rose - Label: LABEL_0
Entity: by - Label: LABEL_0
Entity: 5 - Label: LABEL_0
Entity: % - Label: LABEL_0
Entity: . - Label: LABEL_1
